# Units

Internally, octopus uses atomic units for computing values. Nonetheless, the user can choose the output units with the `inp` parameter [UnitsOutput](https://octopus-code.org/documentation/main/variables/execution/units/unitsoutput/). The default is `atomic` [units], but one can use `ev_angstrom` as well. In the latter case, the spatial dimensions will be measured in Angstrom and the energies will be measured in `eV`. In the `atomic` case, the spatial dimensions will be measured in bohr, while all the other variables will be measured in atomic units. More on this in the [octopus documentation](https://octopus-code.org/documentation/main/variables/execution/units/). The election of the `inp` parameter of the user is reflected in the `parser.log` after the run is finished. We retrieve the unit information from there.

We will compare the benzene and methane test examples.

Note: We will assume that you already know from other tutorials that defining the input file and running octopus in the same notebook is recommended. We will not do it here, we just assume the data exsists in the right folder.

## Methane example (atomic)

For the methane example, we've left the default atomic units, internally Octopus interprets it as a 0:

In [ ]:
cd ../octopus_data/methane/exec

In [ ]:
!head -n 100 parser.log | grep "UnitsOutput"

We load the data:

In [ ]:
import holoviews as hv

from postopus import Run

hv.extension("bokeh", "matplotlib")  # Allow for interactive plots

In [ ]:
cd ../../

In [ ]:
cd methane

In [ ]:
run = Run(".")

In [ ]:
xa = run.default.scf.density("ncdf").isel(step=-1)

Here, we can see that the `xarray` has the units `au`. In the following we will show how to access these units and also the units of the individual coordinates:

In [ ]:
xa.units

In [ ]:
xa.y.units

If we use holoviews for plotting, the coordinates are automatically labeled according to the units of the `xarray`. Unfortunately, labeling the color bar needs to be done manually. For more information about plotting with holoviews, look at the [holoviews]((holoviews_with_postopus.ipynb) tutorial.

In [ ]:
hv_ds = hv.Dataset(xa)
hv_im = hv_ds.to(hv.Image, kdims=["x", "y"], dynamic=True)
hv_im.opts(
    colorbar=True,
    width=500,
    height=400,
    cmap="seismic",
    clabel=f"{xa.name} ({xa.units})",
)

## Benzene example (ev_angstrom)

For the benzene example, we specified `UnitsOutput = ev_angstrom` in the `inp` file. This is interpreted as the number 1 internally in Octopus:

In [ ]:
cd ../benzene/exec

In [ ]:
!head -n 100 parser.log | grep "UnitsOutput"

In [ ]:
cd ..

In [ ]:
runb = Run(".")

We will do the same exploration as we did before with methane:

In [ ]:
xab = runb.default.scf.density(source="ncdf").isel(step=-1)
xab

In [ ]:
xab.units

In [ ]:
xab.x.units

In [ ]:
hv_dsb = hv.Dataset(xab)
hv_imb = hv_dsb.to(hv.Image, kdims=["x", "y"])
hv_imb.opts(
    colorbar=True,
    width=500,
    height=400,
    cmap="seismic",
    clabel=f"{xab.name} ({xab.units})",
)

## Cube format as the exception

The only (known) exception to this rule is the `cube` format. The format itself [specifies that the values should be calculated in atomic units](https://paulbourke.net/dataformats/cube/) (if the number of voxels in a dimension is positive, which to the best of our knowledge, is always the case in Octopus 13.0). Thus, the `cube` output will always be in bohr, independently of what the `UnitsOutput` parameter says.
Actually, `xcrysden` has also an [analogous specification](http://www.xcrysden.org/doc/XSF.html) but is ignored by octopus for now. So, we will read `xcrysden` with the unit that the user specifies in the `inp`, but this [may change in the future](https://gitlab.com/octopus-code/octopus/-/issues/592)

In [ ]:
xa_cube = runb.default.scf.density(source="cube")
xa_cube

In [ ]:
xa_cube.units

In [ ]:
xa_cube.x.units

We will clearly see that the scaling is different in comparison to the last plot, due to the change of units:

In [ ]:
hv_ds_cube = hv.Dataset(xa_cube)
hv_im_cube = hv_ds_cube.to(hv.Image, kdims=["x", "y"])
hv_im_cube.opts(
    colorbar=True,
    width=500,
    height=400,
    cmap="seismic",
    clabel=f"{xa_cube.name} ({xa_cube.units})",
)